In [1]:
import numpy as np
import pandas as pd
import os
from evaluation import *
import random

In [2]:
result='/mnt/user/drive/My Libraries/tutorials&explorations/V1criticality/data/2022-06-26_c_ee11.57199543729056_c_ei10.307919639315172_dt1_results.csv'
df = pd.read_csv(result, header = None)
data = df[1].to_numpy()
time = df[0].to_numpy()

In [3]:
rand_sig = np.random.uniform(-1, 1, time.shape[0])
rand_sig.shape

(1000000,)

In [4]:
df0 = pd.DataFrame({'time':time, 'data':rand_sig})
result_name = '/mnt/user/drive/My Libraries/tutorials&explorations/V1criticality/data/2022-07_15_randm4_results.csv'
df0.to_csv(result_name, mode='a', index=False, header=False)

In [5]:
dfa_analysis(result_name, dt=1, not_remove = True, save_inter=True)

Creating RawArray with float64 data, n_channels=1, n_times=1000000
    Range : 0 ... 999999 =      0.000 ...   999.999 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=1000000
    Range : 0 ... 999999 =      0.000 ...   999.999 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=1000000
    Range : 0 ... 999999 =      0.000 ...   999.999 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=1000000
    Range : 0 ... 999999 =      0.000 ...   999.999 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=1000000
    Range : 0 ... 999999 =      0.000 ...   999.999 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=1000000
    Range : 0 ... 999999 =      0.000 ...   999.999 secs
Ready.


(0.4081313433146249,
 {'delta': 0.5400541215762146,
  'theta': 0.5449977626190278,
  'alpha': 0.49366199567089325,
  'beta': 0.5276907158548721,
  'gamma': 0.4796421657082125,
  'raw': 0.5078950076843392,
  'dfa_this_trial': 0.5078950076843392},
 58.10546875)

### Get the psd of old data

In [7]:
result_name = '/mnt/user/drive/My Libraries/tutorials&explorations/V1criticality/data/2022-06-27_c_ee6_c_ei22_results.csv'
df = pd.read_csv(result_name, header = None)
data = df[1].to_numpy()
peak = get_psd_peak(data, dt=1,save_path=result_name)